<a href="https://colab.research.google.com/github/ridvanyigit/uskdar-vize/blob/main/SayisalG%C3%B6r%C3%BCnt%C3%BCIslemeVize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Kurulumu ve Veri Setinin İndirilmesi (Colab Secrets Yöntemi)

####Sayın Hocam,

Bu not defterinin çalışabilmesi için Kaggle API kimlik bilgilerinizin Colab'e tanıtılması gerekmektedir. Lütfen aşağıdaki 2 adımı izleyiniz:



1.   Colab arayüzünün sol tarafındaki menüden anahtar (🔑) simgesine tıklayarak "Gizli Anahtarlar" (Secrets) bölümünü açın.
2.   Aşağıdaki isim ve değerlere sahip iki adet yeni gizli anahtar oluşturun:

- İsim 1: KAGGLE_USERNAME
  - Değer 1: Sizin Kaggle kullanıcı adınız (Örn: gökalptulum)
- İsim 2: KAGGLE_KEY
  - Değer 2: Sizin Kaggle API anahtarınız (Örn: a1b2c3d4e5f6...)

**Not**: Kaggle API bilgilerinizi kaggle.com sitesindeki `Hesabınız -> Settings -> API bölümünden` "Create New Token" butonuna basarak alabilirsiniz.

In [ ]:
from google.colab import userdata
import os
import zipfile

In [ ]:
# Gizli anahtarları ilgili ortam değişkenlerine atama

print("Colab Secrets'tan Kaggle API bilgileri okunuyor...")
try:
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    print("API bilgileri başarıyla okundu.")
except userdata.SecretNotFoundError:
    print("\nHata: 'KAGGLE_USERNAME' veya 'KAGGLE_KEY' gizli anahtarları bulunamadı.")
    print("Lütfen sol menüdeki anahtar (🔑) ikonundan bu anahtarları oluşturduğunuzdan emin olun.")
    raise SystemExit("Lütfen gizli anahtarları ayarladıktan sonra tekrar deneyin.")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")
    raise SystemExit("Lütfen gizli anahtarları kontrol edin.")

In [ ]:
!pip install -q kaggle

#### Veri setini indirme

In [ ]:
!kaggle datasets download -d nodoubttome/skin-cancer9-classesisic

#### İndirilen dosyaları zipten çıkarma

In [ ]:
print("İndirilen dosyalar 'isic_dataset' klasörüne çıkarılıyor...")
with zipfile.ZipFile('skin-cancer9-classesisic.zip', 'r') as zip_ref:
    zip_ref.extractall('isic_dataset')

print("\nKurulum başarıyla tamamlandı!")

In [ ]:
# Gerekli Kütüphanelerin İçe Aktarılması
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import os
from scipy import ndimage
import warnings
warnings.filterwarnings('ignore')

# VERİ YÜKLEME

In [ ]:
data_path = 'isic_dataset/Skin cancer ISIC The International Skin Imaging Collaboration/Train'


In [ ]:
# Tüm görüntü dosyalarını topla
image_paths = []
labels = []

for class_name in os.listdir(data_path):
    class_path = os.path.join(data_path, class_name)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            if img_name.endswith(('.jpg', '.png', '.jpeg')):
                image_paths.append(os.path.join(class_path, img_name))
                labels.append(class_name)

In [ ]:
# DataFrame oluştur
train_df = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})

In [ ]:
# İlk birkaç satırı göster
print("\n=== VERİ SETİ ÖNİZLEME ===")
print(train_df.head())

In [ ]:
# Toplam görüntü sayısı
print(f"\nToplam görüntü sayısı: {len(train_df)}")
print(f"\nSınıf dağılımı:\n{train_df['label'].value_counts()}")

## Veri Özelliklerinin İncelenmesi

In [ ]:
# Rastgele 100 görüntü örneği üzerinde analiz
sample_size = min(100, len(train_df))
sample_indices = np.random.choice(len(train_df), sample_size, replace=False)

widths = []
heights = []
channels = []
file_sizes = []

for idx in sample_indices:
    img_path = train_df.iloc[idx]['image_path']
    img = cv2.imread(img_path)
    if img is not None:
        h, w, c = img.shape
        heights.append(h)
        widths.append(w)
        channels.append(c)
        file_sizes.append(os.path.getsize(img_path) / 1024)  # KB cinsinden

In [ ]:
# İstatistikler
print(f"\nÇözünürlük İstatistikleri:")
print(f"  Genişlik - Min: {min(widths)}, Max: {max(widths)}, Ort: {np.mean(widths):.1f}")
print(f"  Yükseklik - Min: {min(heights)}, Max: {max(heights)}, Ort: {np.mean(heights):.1f}")
print(f"\nKanal Sayısı: {set(channels)}")
print(f"Dosya Boyutu (KB) - Min: {min(file_sizes):.1f}, Max: {max(file_sizes):.1f}, Ort: {np.mean(file_sizes):.1f}")

In [ ]:
# Görselleştirme
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(widths, bins=20, color='blue', alpha=0.7)
axes[0].set_title('Görüntü Genişlik Dağılımı')
axes[0].set_xlabel('Genişlik (piksel)')
axes[0].set_ylabel('Frekans')

axes[1].hist(heights, bins=20, color='green', alpha=0.7)
axes[1].set_title('Görüntü Yükseklik Dağılımı')
axes[1].set_xlabel('Yükseklik (piksel)')
axes[1].set_ylabel('Frekans')

axes[2].hist(file_sizes, bins=20, color='red', alpha=0.7)
axes[2].set_title('Dosya Boyutu Dağılımı')
axes[2].set_xlabel('Boyut (KB)')
axes[2].set_ylabel('Frekans')

plt.tight_layout()
plt.show()

# GÖRÜNTÜ YÜKLEME VE GÖRSELLEŞTİRME

## Rastgele Görüntüler Seçme

In [ ]:
# Rastgele 9 görüntü seç
random_indices = np.random.choice(len(train_df), 9, replace=False)
selected_images = train_df.iloc[random_indices]

In [ ]:
# Görüntüleri yükle ve göster
fig, axes = plt.subplots(9, 2, figsize=(10, 30))
fig.suptitle('RGB ve Grayscale Görüntüler', fontsize=16, y=0.995)

for i, idx in enumerate(random_indices):
    img_path = train_df.iloc[idx]['image_path']

    # RGB görüntü
    img_rgb = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224, 224))

    # Grayscale görüntü
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    # RGB göster
    axes[i, 0].imshow(img_rgb)
    axes[i, 0].set_title(f'RGB - {train_df.iloc[idx]["label"]}')
    axes[i, 0].axis('off')

    # Grayscale göster
    axes[i, 1].imshow(img_gray, cmap='gray')
    axes[i, 1].set_title(f'Grayscale - {train_df.iloc[idx]["label"]}')
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

## Rastgele Görüntülerin İstatistiksel Özellikleri

In [ ]:
print("\n=== GÖRÜNTÜ İSTATİSTİKLERİ ===")

for i, idx in enumerate(random_indices[:3]):  # İlk 3 görüntü için
    img_path = train_df.iloc[idx]['image_path']

    # RGB
    img_rgb = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224, 224))

    # Grayscale
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    print(f"\n--- Görüntü {i+1} ({train_df.iloc[idx]['label']}) ---")
    print(f"RGB İstatistikleri:")
    print(f"  Min: {img_rgb.min()}, Max: {img_rgb.max()}")
    print(f"  Ortalama: {img_rgb.mean():.2f}, Std: {img_rgb.std():.2f}")

    print(f"Grayscale İstatistikleri:")
    print(f"  Min: {img_gray.min()}, Max: {img_gray.max()}")
    print(f"  Ortalama: {img_gray.mean():.2f}, Std: {img_gray.std():.2f}")

## Histogram Çizimi (RGB + Grayscale)

In [ ]:
print("\n=== HİSTOGRAM ANALİZİ ===")

# İlk 3 görüntü için histogram
for i, idx in enumerate(random_indices[:3]):
    img_path = train_df.iloc[idx]['image_path']

    # Görüntüleri yükle
    img_rgb = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224, 224))
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

    # Histogram çiz
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(f'Histogram Analizi - Görüntü {i+1}', fontsize=14)

    # RGB görüntü
    axes[0, 0].imshow(img_rgb)
    axes[0, 0].set_title('RGB Görüntü')
    axes[0, 0].axis('off')

    # RGB histogram
    colors = ('r', 'g', 'b')
    for j, color in enumerate(colors):
        hist = cv2.calcHist([img_rgb], [j], None, [256], [0, 256])
        axes[0, 1].plot(hist, color=color, label=f'{color.upper()} kanal')
    axes[0, 1].set_title('RGB Histogram')
    axes[0, 1].set_xlabel('Piksel Değeri')
    axes[0, 1].set_ylabel('Frekans')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)

    # Grayscale görüntü
    axes[1, 0].imshow(img_gray, cmap='gray')
    axes[1, 0].set_title('Grayscale Görüntü')
    axes[1, 0].axis('off')

    # Grayscale histogram
    hist_gray = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
    axes[1, 1].plot(hist_gray, color='black')
    axes[1, 1].set_title('Grayscale Histogram')
    axes[1, 1].set_xlabel('Piksel Değeri')
    axes[1, 1].set_ylabel('Frekans')
    axes[1, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Kısa analiz
    print(f"\n--- Histogram Analizi - Görüntü {i+1} ---")
    print("RGB Analizi:")
    print(f"  R kanalı pik: {np.argmax(cv2.calcHist([img_rgb], [0], None, [256], [0, 256]))}")
    print(f"  G kanalı pik: {np.argmax(cv2.calcHist([img_rgb], [1], None, [256], [0, 256]))}")
    print(f"  B kanalı pik: {np.argmax(cv2.calcHist([img_rgb], [2], None, [256], [0, 256]))}")
    print(f"Grayscale pik: {np.argmax(hist_gray)}")

# GÖRÜNTÜ İŞLEME VE İYİLEŞTİRME

In [ ]:
# Örnek görüntü seç
sample_idx = random_indices[0]
img_path = train_df.iloc[sample_idx]['image_path']
img_rgb = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
img_rgb = cv2.resize(img_rgb, (224, 224))
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

## Kontrast Germe (Stretching)

In [ ]:
def contrast_stretching(img):
    """Kontrast germe işlemi"""
    if len(img.shape) == 3:  # RGB
        result = np.zeros_like(img)
        for i in range(3):
            channel = img[:, :, i]
            min_val = channel.min()
            max_val = channel.max()
            result[:, :, i] = ((channel - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        return result
    else:  # Grayscale
        min_val = img.min()
        max_val = img.max()
        return ((img - min_val) / (max_val - min_val) * 255).astype(np.uint8)

In [ ]:
# Kontrast germe uygula
img_rgb_stretched = contrast_stretching(img_rgb)
img_gray_stretched = contrast_stretching(img_gray)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Kontrast Germe Karşılaştırması', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_stretched)
axes[0, 1].set_title('Kontrast Germe RGB')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_stretched, cmap='gray')
axes[1, 1].set_title('Kontrast Germe Grayscale')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Kontrast germe işlemi tamamlandı.")
print("RGB ve grayscale görüntülerde kontrast arttı, detaylar daha belirgin.")

## Histogram Eşitleme

In [ ]:
# RGB için YCrCb dönüşümü ile
img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)
img_ycrcb[:, :, 0] = cv2.equalizeHist(img_ycrcb[:, :, 0])
img_rgb_equalized = cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2RGB)

In [ ]:
# Grayscale için doğrudan
img_gray_equalized = cv2.equalizeHist(img_gray)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Histogram Eşitleme Karşılaştırması', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_equalized)
axes[0, 1].set_title('Histogram Eşitleme RGB')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_equalized, cmap='gray')
axes[1, 1].set_title('Histogram Eşitleme Grayscale')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Histogram eşitleme tamamlandı.")
print("Görüntülerde parlaklık dağılımı daha uniform hale geldi.")

## Gamma Düzeltme

In [ ]:
def gamma_correction(img, gamma):
    """Gamma düzeltme işlemi"""
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype(np.uint8)
    return cv2.LUT(img, table)

gamma_values = [0.5, 1.0, 2.0]

In [ ]:
# RGB için
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Gamma Düzeltme - RGB ve Grayscale', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

for i, gamma in enumerate(gamma_values):
    img_gamma = gamma_correction(img_rgb, gamma)
    axes[0, i+1].imshow(img_gamma)
    axes[0, i+1].set_title(f'RGB Gamma={gamma}')
    axes[0, i+1].axis('off')

# Grayscale için
axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

for i, gamma in enumerate(gamma_values):
    img_gamma = gamma_correction(img_gray, gamma)
    axes[1, i+1].imshow(img_gamma, cmap='gray')
    axes[1, i+1].set_title(f'Grayscale Gamma={gamma}')
    axes[1, i+1].axis('off')

plt.tight_layout()
plt.show()

print("Gamma düzeltme tamamlandı.")
print("Gamma < 1: Görüntü aydınlandı")
print("Gamma = 1: Değişiklik yok")
print("Gamma > 1: Görüntü koyulaştı")

# GÜRÜLTÜ AZALTMA

## Median Blur Uygulama

In [ ]:
img_rgb_median = cv2.medianBlur(img_rgb, 5)
img_gray_median = cv2.medianBlur(img_gray, 5)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Median Blur Karşılaştırması', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_median)
axes[0, 1].set_title('Median Blur RGB')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_median, cmap='gray')
axes[1, 1].set_title('Median Blur Grayscale')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## Gaussian Blur Uygulama

In [ ]:
img_rgb_gaussian = cv2.GaussianBlur(img_rgb, (5, 5), 0)
img_gray_gaussian = cv2.GaussianBlur(img_gray, (5, 5), 0)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Gaussian Blur Karşılaştırması', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_gaussian)
axes[0, 1].set_title('Gaussian Blur RGB')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_gaussian, cmap='gray')
axes[1, 1].set_title('Gaussian Blur Grayscale')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Karşılaştırma
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Blur Yöntemleri Karşılaştırması (Grayscale)', fontsize=14)

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Orijinal')
axes[0].axis('off')

axes[1].imshow(img_gray_median, cmap='gray')
axes[1].set_title('Median Blur\n(Kenarları korur)')
axes[1].axis('off')

axes[2].imshow(img_gray_gaussian, cmap='gray')
axes[2].set_title('Gaussian Blur\n(Daha yumuşak)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("Median blur: Kenarları daha iyi korur, tuz-biber gürültüsüne etkili")
print("Gaussian blur: Daha yumuşak görüntü, bazı detay kaybı var")

# DÖNDÜRME VE AYNA ÇEVİRME

## Rastgele Döndürme

In [ ]:
# 0-10 derece arası rastgele açı
angle = np.random.uniform(0, 10)
print(f"Döndürme açısı: {angle:.2f} derece")

In [ ]:
# RGB döndür
h, w = img_rgb.shape[:2]
M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
img_rgb_rotated = cv2.warpAffine(img_rgb, M, (w, h))

In [ ]:
# Grayscale döndür
img_gray_rotated = cv2.warpAffine(img_gray, M, (w, h))

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle(f'Rastgele Döndürme ({angle:.2f}°)', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_rotated)
axes[0, 1].set_title('Döndürülmüş RGB')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_rotated, cmap='gray')
axes[1, 1].set_title('Döndürülmüş Grayscale')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## Yatay Ayna Çevirme

In [ ]:
# Yatay flip
img_rgb_flipped = cv2.flip(img_rgb, 1)
img_gray_flipped = cv2.flip(img_gray, 1)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Yatay Ayna Çevirme', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_flipped)
axes[0, 1].set_title('Flip RGB')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_flipped, cmap='gray')
axes[1, 1].set_title('Flip Grayscale')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Görüntüler yatay eksende ayna çevrildi.")
print("Simetri farklılıkları gözlemlendi.")

# FREKANS ALANINDA FİLTRELEME (FFT)

## Fourier Dönüşümü

In [ ]:
# Grayscale görüntü üzerinde FFT
f_transform = np.fft.fft2(img_gray)
f_shift = np.fft.fftshift(f_transform)

In [ ]:
# Magnitude spektrum
magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Fourier Dönüşümü', fontsize=14)

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Orijinal Grayscale')
axes[0].axis('off')

axes[1].imshow(magnitude_spectrum, cmap='gray')
axes[1].set_title('Frekans Spektrumu')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("FFT uygulandı. Merkezdeki yüksek yoğunluk düşük frekansları gösterir.")

## Alçak Geçiren Filtre Uygulama

In [ ]:
# Alçak geçiren filtre maskesi oluştur
rows, cols = img_gray.shape
crow, ccol = rows // 2, cols // 2

In [ ]:
# Maske - merkez beyaz, kenarlar siyah
mask = np.zeros((rows, cols), np.uint8)
r = 30  # Yarıçap
center = [crow, ccol]
x, y = np.ogrid[:rows, :cols]
mask_area = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= r*r
mask[mask_area] = 1

In [ ]:
# Maskeyi uygula
f_shift_filtered = f_shift * mask

In [ ]:
# Görselleştir
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(mask, cmap='gray')
axes[0].set_title('Alçak Geçiren Filtre Maskesi')
axes[0].axis('off')

magnitude_filtered = 20 * np.log(np.abs(f_shift_filtered) + 1)
axes[1].imshow(magnitude_filtered, cmap='gray')
axes[1].set_title('Filtrelenmiş Frekans Spektrumu')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Ters Fourier (Inverse FFT)

In [ ]:
# Ters FFT
f_ishift = np.fft.ifftshift(f_shift_filtered)
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('FFT Filtreleme Süreci', fontsize=14)

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Orijinal')
axes[0].axis('off')

axes[1].imshow(magnitude_spectrum, cmap='gray')
axes[1].set_title('Frekans Spektrumu')
axes[1].axis('off')

axes[2].imshow(img_back, cmap='gray')
axes[2].set_title('Filtrelenmiş Görüntü\n(Alçak geçiren)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Karşılaştırma

In [ ]:
print("\n=== FFT KARŞILAŞTIRMA ANALİZİ ===")
print("Alçak geçiren filtre yüksek frekansları (detayları) azalttı.")
print("Görüntü daha yumuşak ve bulanık hale geldi.")
print("Kenarlar ve ince detaylar kayboldu.")

# Detaylı karşılaştırma
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('FFT Filtreleme Detaylı Karşılaştırma', fontsize=14)

axes[0, 0].imshow(img_gray, cmap='gray')
axes[0, 0].set_title('Orijinal Grayscale')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_back, cmap='gray')
axes[0, 1].set_title('FFT Filtrelenmiş')
axes[0, 1].axis('off')

# Fark görüntüsü
diff = np.abs(img_gray.astype(float) - img_back.astype(float))
axes[1, 0].imshow(diff, cmap='hot')
axes[1, 0].set_title('Fark (Kayıp Detaylar)')
axes[1, 0].axis('off')

# Histogram karşılaştırma
axes[1, 1].hist(img_gray.ravel(), bins=256, alpha=0.5, label='Orijinal', color='blue')
axes[1, 1].hist(img_back.ravel(), bins=256, alpha=0.5, label='Filtrelenmiş', color='red')
axes[1, 1].set_title('Histogram Karşılaştırma')
axes[1, 1].set_xlabel('Piksel Değeri')
axes[1, 1].set_ylabel('Frekans')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# KESKİNLEŞTİRME VE ENTERPOLASYON

## Unsharp Masking ile Keskinleştirme

In [ ]:
def unsharp_mask(img, sigma=1.0, strength=1.5):
    """Unsharp masking keskinleştirme"""
    # Gaussian blur uygula
    blurred = cv2.GaussianBlur(img, (0, 0), sigma)
    # Keskinleştirilmiş görüntü
    sharpened = cv2.addWeighted(img, 1.0 + strength, blurred, -strength, 0)
    return sharpened

In [ ]:
# RGB keskinleştirme
img_rgb_sharp = unsharp_mask(img_rgb, sigma=1.0, strength=1.5)

In [ ]:
# Grayscale keskinleştirme
img_gray_sharp = unsharp_mask(img_gray, sigma=1.0, strength=1.5)

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Unsharp Masking Keskinleştirme', fontsize=14)

axes[0, 0].imshow(img_rgb)
axes[0, 0].set_title('Orijinal RGB')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_sharp)
axes[0, 1].set_title('Keskinleştirilmiş RGB')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray, cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_sharp, cmap='gray')
axes[1, 1].set_title('Keskinleştirilmiş Grayscale')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("Unsharp masking uygulandı.")
print("Kenarlar ve detaylar daha belirgin hale geldi.")
print("RGB ve grayscale'de benzer keskinleştirme etkisi görüldü.")

In [ ]:
# Detay karşılaştırması için zoom
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Keskinleştirme Detay Karşılaştırması (Yakınlaştırılmış)', fontsize=14)

# Merkez bölgeyi al
h, w = img_gray.shape
crop_y, crop_x = h//3, w//3
crop_h, crop_w = h//3, w//3

axes[0, 0].imshow(img_rgb[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w])
axes[0, 0].set_title('Orijinal RGB (Detay)')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_sharp[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w])
axes[0, 1].set_title('Keskinleştirilmiş RGB (Detay)')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w], cmap='gray')
axes[1, 0].set_title('Orijinal Grayscale (Detay)')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_sharp[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w], cmap='gray')
axes[1, 1].set_title('Keskinleştirilmiş Grayscale (Detay)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## Bicubic Enterpolasyon

In [ ]:
# 2 kat büyüt
scale_factor = 2
new_width = img_rgb.shape[1] * scale_factor
new_height = img_rgb.shape[0] * scale_factor

In [ ]:
# RGB bicubic interpolation
img_rgb_sharp_upscaled = cv2.resize(
    img_rgb_sharp,
    (new_width, new_height),
    interpolation=cv2.INTER_CUBIC
    )

In [ ]:
# Grayscale bicubic interpolation
img_gray_sharp_upscaled = cv2.resize(
    img_gray_sharp,
    (new_width, new_height),
    interpolation=cv2.INTER_CUBIC
    )

In [ ]:
print(f"Orijinal boyut: {img_rgb.shape[:2]}")
print(f"Yeni boyut: {img_rgb_sharp_upscaled.shape[:2]}")

In [ ]:
# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle('Bicubic Enterpolasyon (2x Büyütme)', fontsize=14)

axes[0, 0].imshow(img_rgb_sharp)
axes[0, 0].set_title(f'Keskinleştirilmiş RGB\n{img_rgb_sharp.shape[:2]}')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_sharp_upscaled)
axes[0, 1].set_title(f'Büyütülmüş RGB\n{img_rgb_sharp_upscaled.shape[:2]}')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_gray_sharp, cmap='gray')
axes[1, 0].set_title(f'Keskinleştirilmiş Grayscale\n{img_gray_sharp.shape[:2]}')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_gray_sharp_upscaled, cmap='gray')
axes[1, 1].set_title(f'Büyütülmüş Grayscale\n{img_gray_sharp_upscaled.shape[:2]}')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## Farklı enterpolasyon yöntemleri karşılaştırması

In [ ]:
# Farklı yöntemler
img_nearest = cv2.resize(
    img_gray_sharp,
    (new_width, new_height),
    interpolation=cv2.INTER_NEAREST
    )

img_linear = cv2.resize(
    img_gray_sharp,
    (new_width, new_height),
    interpolation=cv2.INTER_LINEAR
    )

img_cubic = cv2.resize(
    img_gray_sharp,
    (new_width, new_height),
    interpolation=cv2.INTER_CUBIC
    )

img_lanczos = cv2.resize(
    img_gray_sharp,
    (new_width, new_height),
    interpolation=cv2.INTER_LANCZOS4
    )

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle('Farklı Enterpolasyon Yöntemleri Karşılaştırması', fontsize=14)

axes[0, 0].imshow(img_nearest, cmap='gray')
axes[0, 0].set_title('Nearest Neighbor\n(En hızlı, en düşük kalite)')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_linear, cmap='gray')
axes[0, 1].set_title('Bilinear\n(Orta hız, orta kalite)')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_cubic, cmap='gray')
axes[1, 0].set_title('Bicubic\n(Daha yavaş, daha iyi kalite)')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_lanczos, cmap='gray')
axes[1, 1].set_title('Lanczos\n(En yavaş, en iyi kalite)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nEnterpolasyon kalite sıralaması:")
print("1. Lanczos: En iyi kalite, en az bulanıklık")
print("2. Bicubic: İyi kalite, kabul edilebilir hız")
print("3. Bilinear: Orta kalite, hızlı")
print("4. Nearest: Düşük kalite, pikselleşme var, çok hızlı")

In [ ]:
# Detaylı kalite karşılaştırması
crop_size = 100
cy, cx = new_height//2, new_width//2

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Enterpolasyon Kalite Detayları (Yakınlaştırılmış)', fontsize=14)

axes[0, 0].imshow(img_nearest[cy:cy+crop_size, cx:cx+crop_size], cmap='gray')
axes[0, 0].set_title('Nearest (Detay)')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_linear[cy:cy+crop_size, cx:cx+crop_size], cmap='gray')
axes[0, 1].set_title('Bilinear (Detay)')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_cubic[cy:cy+crop_size, cx:cx+crop_size], cmap='gray')
axes[1, 0].set_title('Bicubic (Detay)')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_lanczos[cy:cy+crop_size, cx:cx+crop_size], cmap='gray')
axes[1, 1].set_title('Lanczos (Detay)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

# GENEL ÖZET VE SONUÇLAR

In [ ]:
print("\n" + "="*80)
print("GENEL DEĞERLENDİRME")
print("="*80)

print("\n1. VERİ SETİ ÖZETİ:")
print(f"   - Toplam görüntü: {len(train_df)}")
print(f"   - Sınıf sayısı: {train_df['label'].nunique()}")
print(f"   - Ortalama çözünürlük: {np.mean(widths):.0f}x{np.mean(heights):.0f}")

print("\n2. UYGULANAN İŞLEMLER:")
print("   ✓ RGB ve Grayscale görüntü yükleme")
print("   ✓ Histogram analizi ve görselleştirme")
print("   ✓ Kontrast germe (stretching)")
print("   ✓ Histogram eşitleme")
print("   ✓ Gamma düzeltme (0.5, 1.0, 2.0)")
print("   ✓ Median ve Gaussian blur")
print("   ✓ Döndürme ve flip işlemleri")
print("   ✓ FFT ve alçak geçiren filtre")
print("   ✓ Unsharp masking keskinleştirme")
print("   ✓ Bicubic enterpolasyon (2x büyütme)")

print("\n3. ÖNEMLİ BULGULAR:")
print("   • RGB histogramları genelde benzer dağılımlı, deri tonu baskın")
print("   • Kontrast germe detayları belirginleştirdi")
print("   • Histogram eşitleme parlaklık dağılımını iyileştirdi")
print("   • Median blur kenarları Gaussian'dan daha iyi korudu")
print("   • FFT alçak geçiren filtre detayları azalttı (beklenen)")
print("   • Unsharp masking kenarları başarıyla keskinleştirdi")
print("   • Bicubic enterpolasyon kaliteli büyütme sağladı")

print("\n4. RGB vs GRAYSCALE KARŞILAŞTIRMA:")
print("   • RGB: Daha fazla bilgi, renk kanalları ayrı işlenebilir")
print("   • Grayscale: Daha hızlı işlem, daha az bellek")
print("   • Sonuçlar genel olarak benzer ama RGB daha zengin")
print("   • FFT ve bazı filtreler grayscale'de daha verimli")

print("\n5. ÖNERİLER:")
print("   • Deri lezyonu sınıflandırma için RGB önerilir")
print("   • Ön işlemede histogram eşitleme + keskinleştirme etkili")
print("   • Gürültü varsa median blur tercih edilmeli")
print("   • Veri artırma için döndürme ve flip kullanılabilir")

In [ ]:
import os
os.makedirs('outputs', exist_ok=True)
counter = 1

original_show = plt.show
def custom_show():
    global counter
    plt.savefig(f'outputs/img_{counter:02d}.png', dpi=300, bbox_inches='tight')
    counter += 1
    original_show()
plt.show = custom_show